In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
import gradio as gr 

In [3]:
# Load environment variables from .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found")
elif api_key[:8]!= "sk-proj-":
    print("An API Key was found, but it doesn't start with 'sk-proj-', please check you're using right api key")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
# Generic system message

system_message = "You are a helpful assistant"

In [5]:
openai = OpenAI()

In [6]:
# Lets wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role":"system", "content":system_message},
        {"role":"user", "content":prompt}
    ]
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
        max_tokens= 500,
    )
    return response.choices[0].message.content

In [7]:
message_gpt("What is today's date?")     # still not great with current events

"Today's date is October 3, 2023."

Lets build the User Interface with Gradio

In [14]:
# simple function

def shout(text):
    print(f"The function shout has been called with the text : {text}")
    return text.upper()

The function shout has been called with the text : {text}
The function shout has been called with the text : {text}
The function shout has been called with the text : {text}


In [9]:
shout('hello, welcome to gradio!')

'HELLO, WELCOME TO GRADIO!'

Gradio can take single as well as multiple inputs and outputs 

In [10]:
view = gr.Interface(fn=shout, inputs="textbox", outputs="textbox")
view.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [11]:
view = gr.Interface(fn=shout, inputs="textbox", outputs="textbox", allow_flagging="never")     #not to create flagged folder
view. launch(share=True)

c:\Users\KajalMore\VSCODE\self_learning\env\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


The user interface is served off as a public server (url) but the function is running on your local machine, which is amazing with gradio

In [15]:
view = gr.Interface(
    fn = shout,
    inputs= [gr.Textbox(label="Your message :", lines = 6)],
    outputs= [gr.Textbox(label="Response :", lines=8)],
    allow_flagging= "never"
)
view.launch()

c:\Users\KajalMore\VSCODE\self_learning\env\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


The function shout has been called with the text : hey


Now will assign gpt input prompt as a input function to gradio and the gpt response will come as a output of gradio interface

In [16]:
view = gr.Interface(
    fn = message_gpt,
    inputs= [gr.Textbox(label="Your message :", lines = 6)],
    outputs= [gr.Textbox(label="Response :", lines=8)],
    allow_flagging= "never"
)
view.launch()

c:\Users\KajalMore\VSCODE\self_learning\env\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Will try to show the response generated in a Markdown

In [18]:
system_message = "You are a helpful assistant that responds in markdown."

view = gr.Interface(
    fn = message_gpt,
    inputs= [gr.Textbox(label="Your message :")],
    outputs= [gr.Markdown(label="Response :")],
    allow_flagging= "never"
)
view.launch()

c:\Users\KajalMore\VSCODE\self_learning\env\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Lets stream the response with gradio. Take a note that while streaming the output response with gradio we can not stream back chunk by chunk rather we stream back the entire commulative result with the yield generator

In [19]:
# Lets wrap a call to GPT-4o-mini in a simple function

def stream_gpt(prompt):
    messages = [
        {"role":"system", "content":system_message},
        {"role":"user", "content":prompt}
    ]
    stream_response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
        max_tokens= 256,
        stream= True
    )
    result = ""
    for chunk in stream_response:
        result += chunk.choices[0].delta.content or ""
        yield result


In [20]:
view = gr.Interface(
    fn = stream_gpt,
    inputs= [gr.Textbox(label="Your message :")],
    outputs= [gr.Markdown(label="Response :")],
    allow_flagging= "never"
)
view.launch()

c:\Users\KajalMore\VSCODE\self_learning\env\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
